# MEVN stack experiment with Jupyter

In this notebook I want to play with Jupyter and show the steps of how to create a MEVN application from a notebook. Normally I would do this in the normal Linux terminal and a text editor, but since we can combine code, explanation and shell commands, I want to create a story in this notebook which hopefully will be of any help of people experimenting with full-stack development. I will create the application, use some simple Linux tricks and use Selenium to test the application.

Please note development like this is far from optimal, but I think it is very cool what you can achieve without leaving your notebook, knowing a bit of Docker, Linux and Python.


## Objective
Setup an application with the following elements

* [MongoDB](https://www.mongodb.com/)
* [ExpressJS](https://expressjs.com/)
* [VueJS](https://vuejs.org/)
* [NodeJS](https://nodejs.org/en/)

using

* [Jupyter](http://jupyter.org/) notebook
* [Docker](https://www.docker.com/)
* [Scrapy](https://scrapy.org/)
* [PyMongo](https://api.mongodb.com/python/current/)
* [Selenium](http://www.seleniumhq.org/)

<img class="logo" src="logos/jupyter.png" />

### Notebook settings

In [2]:
%%html
<style>
img.logo{
    height: 100px;
}

img.screenshot{
    max-width:500px; 
    -webkit-filter: drop-shadow(5px 5px 5px #222);
    filter: drop-shadow(2px 5px 5px #222);
    margin: 50px auto;
}
</style>

Clean up the images from earlier runs.

In [3]:
rm *.png

rm: *.png: No such file or directory


In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
!jupyter --version

4.2.1


In [6]:
!jupyter notebook --version

4.3.1


<img class="logo" src="logos/python.PNG"/>
### Python version

In [7]:
import platform
platform.python_version()

'3.6.0'

<img class="logo" src="logos/docker.PNG"/>

## Docker setup
### Docker version

In [8]:
!docker version

Client:
 Version:      17.09.1-ce
 API version:  1.32
 Go version:   go1.8.3
 Git commit:   19e2cf6
 Built:        Thu Dec  7 22:22:25 2017
 OS/Arch:      darwin/amd64

Server:
 Version:      17.09.1-ce
 API version:  1.32 (minimum version 1.12)
 Go version:   go1.8.3
 Git commit:   19e2cf6
 Built:        Thu Dec  7 22:28:28 2017
 OS/Arch:      linux/amd64
 Experimental: true


In [9]:
!docker ps

CONTAINER ID        IMAGE                      COMMAND             CREATED             STATUS              PORTS                    NAMES
4764e63d228b        jitsejan/pelican-python3   "fab reserve"       29 minutes ago      Up 30 minutes       0.0.0.0:8093->8093/tcp   pelicanblog_pelican_1


### Docker client
#### Setup the client

In [11]:
!pip install docker

    100% |████████████████████████████████| 122kB 307kB/s ta 0:00:01
    100% |████████████████████████████████| 204kB 513kB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 2.8MB/s ta 0:00:011
    100% |████████████████████████████████| 61kB 4.1MB/s eta 0:00:01
    100% |████████████████████████████████| 337kB 1.5MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 1.7MB/s eta 0:00:01
    100% |████████████████████████████████| 143kB 1.8MB/s ta 0:00:01
  Found existing installation: idna 2.2
    Uninstalling idna-2.2:
      Successfully uninstalled idna-2.2
  Found existing installation: urllib3 1.20
    Uninstalling urllib3-1.20:
      Successfully uninstalled urllib3-1.20
  Found existing installation: chardet 2.3.0
    Uninstalling chardet-2.3.0:
      Successfully uninstalled chardet-2.3.0
  Found existing installation: requests 2.12.4
    Uninstalling requests-2.12.4:
      Successfully uninstalled requests-2.12.4


In [12]:
import docker
docker_client = docker.from_env()

#### Available containers

In [13]:
for cntr in docker_client.containers.list():
    print("name={} (id={})".format(cntr.name, cntr.id))

name=pelicanblog_pelican_1 (id=4764e63d228b8cfcfc71eba449cb3036ac5e14292facc096e4d16dc3e6329d3b)


<img class="logo" src="logos/mongodb.PNG"/>

## MongoDB

Check if the Mongo Docker container is running, otherwise, start the container.

In [14]:
mongo_running = False
for cntr in docker_client.containers.list():
    if 'mongo' in cntr.attrs['Config']['Image']:
        mongo_running = True
        container = cntr
if mongo_running is False:
    container = docker_client.containers.run("mongo:latest", name='mongo', ports={'27017': '27017'}, detach=True)

In [15]:
container

<Container: 77f8450c94>

Verification that the Mongo container is running:

In [16]:
!docker ps | grep mongo

77f8450c94fb        mongo:latest               "docker-entrypoint..."   Less than a second ago   Up Less than a second   0.0.0.0:27017->27017/tcp   mongo


See [Documentation](https://api.mongodb.com/python/current/installation.html) to install PyMongo.

```shell
$ pip install pymongo
```

In [17]:
from pymongo import MongoClient
mongo_client = MongoClient('localhost', 27017)

<img class="logo" src="logos/scrapy.PNG"/>

## Data gathering

Create the Scrapy pipeline to write the scraping results to MongoDB.

In [19]:
import pymongo

class MongoPipeline(object):

    collection_name = 'games'

    def __init__(self, mongo_uri, mongo_db):
        self.mongo_uri = mongo_uri
        self.mongo_db = mongo_db

    @classmethod
    def from_crawler(cls, crawler):
        return cls(
            mongo_uri=crawler.settings.get('MONGO_URI'),
            mongo_db=crawler.settings.get('MONGO_DATABASE', 'items')
        )

    def open_spider(self, spider):
        self.client = pymongo.MongoClient(self.mongo_uri)
        self.db = self.client[self.mongo_db]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.db[self.collection_name].insert_one(dict(item))
        return item

Retrieving data from https://www.nintendo.co.jp/ir/en/finance/software/index.html with the following markup:

```html
<ul class="sales_layout">
    <li class="sales_layout_list">
        <div class="ta_l">
            <p class="sales_title">The Legend of Zelda:<br> Breath of the Wild</p>
            <p class="sales_value"><span>4.70</span> million pcs.</p>
        </div>
        <div class="ta_r">
            <p>
                <img src="./img/data_switch_001.png" alt="" width="110" height="" class="sales_product1">
            </p>
        </div>
    </li>
</ul>
```

Using Scrapy we can gather the data in a convenient way.

In [20]:
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
import re

class ConsoleSpider(scrapy.Spider):
    name = "games"
    start_urls = [
        "https://www.nintendo.co.jp/ir/en/finance/software/index.html",
        "https://www.nintendo.co.jp/ir/en/finance/software/wiiu.html",
        "https://www.nintendo.co.jp/ir/en/finance/software/3ds.html",
        "https://www.nintendo.co.jp/ir/en/finance/software/wii.html",
        "https://www.nintendo.co.jp/ir/en/finance/software/ds.html"
    ]
    custom_settings = {
        'LOG_LEVEL': logging.CRITICAL,
        'DOWNLOAD_DELAY': .25,
        'RANDOMIZE_DOWNLOAD_DELAY': True,
        'ITEM_PIPELINES': {
            '__main__.MongoPipeline': 300,
        },
        'MONGO_URI': 'mongodb://localhost:27017',
        'MONGO_DATABASE': 'nintendo'
    }

    def parse(self, response):
         for cons in response.css('li.sales_layout_list'):
            yield {
                'console': response.css('div.tab div.tabInner span::text').extract_first(),
                'name': cons.css('p.sales_title::text').extract_first().strip(),
                'image': 'https://www.nintendo.co.jp/ir/en/finance/software' + cons.css('p img::attr(src)').extract_first()[1:],
                'sales': cons.css('p.sales_value span::text').extract()[0].strip()
            }

In [21]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(ConsoleSpider)
process.start()

2018-01-15 01:04:34 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2018-01-15 01:04:34 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2018-01-15 01:04:34 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2018-01-15 01:04:35 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.Redirec

2018-01-15 01:04:36 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.nintendo.co.jp/ir/en/finance/software/wiiu.html> (referer: None)
2018-01-15 01:04:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.nintendo.co.jp/ir/en/finance/software/wiiu.html>
{'console': 'Wii U', 'name': 'Mario Kart 8', 'image': 'https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_mariokart8.png', 'sales': '8.38'}
2018-01-15 01:04:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.nintendo.co.jp/ir/en/finance/software/wiiu.html>
{'console': 'Wii U', 'name': 'New', 'image': 'https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_newmariou.png', 'sales': '5.73'}
2018-01-15 01:04:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.nintendo.co.jp/ir/en/finance/software/wiiu.html>
{'console': 'Wii U', 'name': 'Super Mario 3D World', 'image': 'https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_mario3dworld.png', 'sales': '5.70'}
2018-01-15 01

#### Verify the Nintendo database is created

In [22]:
if 'nintendo' in mongo_client.database_names():
    print('Database found!')

Database found!


#### Verify the games collection is created inside the Nintendo database

In [23]:
db = mongo_client['nintendo']
if 'games' in db.collection_names():
    print('Collection found!')

Collection found!


#### Retrieve the games from the collection

In [24]:
games = list(db['games'].find({}))

In [25]:
print("Found {} games".format(len(games)))

Found 45 games


## Retrieve the images
#### Delete the old images

In [26]:
!rm -rf images/*

In [27]:
import re
updated_games = []
for gameindex, game in enumerate(games):
    image = game['image']
    image_short = image.split('/')[-1]
    !wget --no-check-certificate $image -P images/ 
    game['image'] = 'images/' + image_short
    game.pop('_id', None)
    updated_games.append(game)

--2018-01-15 01:04:51--  https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_mariokart8.png
Resolving www.nintendo.co.jp... 23.72.149.166
Connecting to www.nintendo.co.jp|23.72.149.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41726 (41K) [image/png]
Saving to: 'images/data_wiiu_mariokart8.png'

data_wiiu_mariokart 100%[===================>]  40.75K   244KB/s    in 0.2s    

2018-01-15 01:04:52 (244 KB/s) - 'images/data_wiiu_mariokart8.png' saved [41726/41726]



ObjectId('5a5bf0140a4693652827dc62')

--2018-01-15 01:04:52--  https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_newmariou.png
Resolving www.nintendo.co.jp... 23.72.149.166
Connecting to www.nintendo.co.jp|23.72.149.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39007 (38K) [image/png]
Saving to: 'images/data_wiiu_newmariou.png'

data_wiiu_newmariou 100%[===================>]  38.09K   238KB/s    in 0.2s    

2018-01-15 01:04:53 (238 KB/s) - 'images/data_wiiu_newmariou.png' saved [39007/39007]



ObjectId('5a5bf0140a4693652827dc63')

--2018-01-15 01:04:53--  https://www.nintendo.co.jp/ir/en/finance/software/img/data_wiiu_mario3dworld.png
Resolving www.nintendo.co.jp... 23.72.149.166
Connecting to www.nintendo.co.jp|23.72.149.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40182 (39K) [image/png]
Saving to: 'images/data_wiiu_mario3dworld.png'

data_wiiu_mario3dwo 100%[===================>]  39.24K  81.1KB/s    in 0.5s    

2018-01-15 01:04:54 (81.1 KB/s) - 'images/data_wiiu_mario3dworld.png' saved [40182/40182]



...

ObjectId('5a5bf0150a4693652827dc8c')

--2018-01-15 01:05:49--  https://www.nintendo.co.jp/ir/en/finance/softwaremg/data_ds_010.jpg
Resolving www.nintendo.co.jp... 23.72.149.166
Connecting to www.nintendo.co.jp|23.72.149.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-01-15 01:05:50 ERROR 404: Not Found.



ObjectId('5a5bf0150a4693652827dc8d')

--2018-01-15 01:05:50--  https://www.nintendo.co.jp/ir/en/finance/softwaremg/data_ds_009.jpg
Resolving www.nintendo.co.jp... 23.72.149.166
Connecting to www.nintendo.co.jp|23.72.149.166|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2018-01-15 01:05:51 ERROR 404: Not Found.



ObjectId('5a5bf0150a4693652827dc8e')

#### Verify the downloaded images
Run the `ls images` to show the available images and save the list to `lsimages`.

In [28]:
lsimages = !ls images
lsimages

['data_3ds_001.jpg',
 'data_3ds_002.jpg',
 'data_3ds_003.jpg',
 'data_3ds_004.jpg',
 'data_3ds_009.jpg',
 'data_3ds_pokemonoras.jpg',
 'data_3ds_pokemonsunmoon.jpg',
 'data_3ds_pokemonxy.jpg',
 'data_3ds_smashbros.jpg',
 'data_3ds_tomodachicollection.jpg',
 'data_ds_001.jpg',
 'data_ds_002.jpg',
 'data_ds_006.jpg',
 'data_ds_007.jpg',
 'data_ds_008.jpg',
 'data_switch_001.png',
 'data_switch_002.png',
 'data_switch_003.png',
 'data_switch_004.png',
 'data_switch_005.png',
 'data_wii_001.jpg',
 'data_wii_002.jpg',
 'data_wii_003.jpg',
 'data_wii_004.jpg',
 'data_wii_005.jpg',
 'data_wii_006.jpg',
 'data_wii_007.jpg',
 'data_wii_008.jpg',
 'data_wii_009.jpg',
 'data_wii_wiiparty.jpg',
 'data_wiiu_mario3dworld.png',
 'data_wiiu_mariokart8.png',
 'data_wiiu_marioparty10.jpg',
 'data_wiiu_newmariou.png',
 'data_wiiu_newsuperluigiu.png',
 'data_wiiu_nintendoland.png',
 'data_wiiu_smashbros.png',
 'data_wiiu_splatoon.jpg',
 'data_wiiu_supermariomaker.png',
 'data_wiiu_zeldahd.png']

#### Show the first image

In [29]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url='images/'+lsimages.list[0])

#### Delete the old games from the database

In [30]:
result = db.games.delete_many({})

#### Insert the updated games

In [31]:
result = db.games.insert_many(updated_games)

<img class="logo" src="logos/express.PNG" />
## Back-end
Create the API with [ExpressJS](https://expressjs.com/) and [Mongoose](http://mongoosejs.com/).

First make sure NodeJS is installed.

In [32]:
!node --version

v8.4.0


In [33]:
!npm --version

5.6.0


### Create Node package and install Express generator
Install the ExpressJS [generator](https://expressjs.com/en/starter/generator.html) after creating the `server` folder.

In [34]:
!rm -rf server

In [35]:
!mkdir server && cd server && npm init -y && npm install express-generator --save

Wrote to /Users/jitsejan/code/pelican-blog/content/notebooks/server/package.json:

{
  "name": "server",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}


npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN server@1.0.0 No description
npm WARN server@1.0.0 No repository field.

+ express-generator@4.15.5
added 6 packages in 3.999s


#### Check ExpressJS version

In [36]:
!cd server && npm express --version

5.6.0


#### Create the scaffold for the server
Note that we use `npx` instead of `npm` because we run NPM from a <b>local</b> folder.

In [37]:
!cd server && npx express -y --force --git --view ejs .


   create : .
   create : ./package.json
   create : ./app.js
   create : ./.gitignore
   create : ./public
   create : ./routes
   create : ./routes/index.js
   create : ./routes/users.js
   create : ./views
   create : ./views/index.ejs
   create : ./views/error.ejs
   create : ./bin
   create : ./bin/www
   create : ./public/javascripts
   create : ./public/images
   create : ./public/stylesheets
   create : ./public/stylesheets/style.css

   install dependencies:
     $ cd . && npm install

   run the app:
     $ DEBUG=server:* npm start



In [38]:
ls server/

app.js             node_modules/      package.json       routes/
bin/               package-lock.json  public/            views/


In [39]:
!cat server/package.json

{
  "name": "server",
  "version": "0.0.0",
  "private": true,
  "scripts": {
    "start": "node ./bin/www"
  },
  "dependencies": {
    "body-parser": "~1.18.2",
    "cookie-parser": "~1.4.3",
    "debug": "~2.6.9",
    "ejs": "~2.5.7",
    "express": "~4.15.5",
    "morgan": "~1.9.0",
    "serve-favicon": "~2.4.5"
  }
}


#### Install dependencies

In [40]:
!cd server && npm install

added 57 packages and removed 5 packages in 5.541s32minfo lifecycle server@0.0.0~prepare: ser


#### Run the app

In [41]:
import subprocess
proc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)
proc.pid

95010

#### Request the ExpressJS app

In [42]:
import requests
import time
from IPython.core.display import display, HTML

time.sleep(5)
resp = requests.get('http://localhost:3000')
display(HTML(resp.text))

2018-01-15 01:07:08 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): localhost
2018-01-15 01:07:08 [urllib3.connectionpool] DEBUG: http://localhost:3000 "GET / HTTP/1.1" 200 207


#### Stop the ExpressJS app
Use `fuser` to find the process that uses port 3000 and kill it.

In [50]:
process=!lsof -i:3000 -t
expressid = int(process[0])
expressid

95013

In [51]:
!kill -9 $expressid

#### Check if the server is down

In [52]:
try:
    resp = requests.get('http://localhost:3000')
except:
    print("Server is down!")

2018-01-15 01:11:04 [urllib3.connectionpool] DEBUG: Starting new HTTP connection (1): localhost


Server is down!


<img class="logo" src="logos/mongoose.PNG" />
## Connection from ExpressJS to MongoDB 
[Mongoose](http://mongoosejs.com/) will be used to connect the Node application to the database.

#### Install package

In [53]:
!cd server && npm install --save mongoose

+ mongoose@4.13.9m░░░░░░⸩ ⠧ postinstall: info lifecycle mongoose@4.13.9~postinstall:
added 28 packages in 6.84s


#### Check installation

In [54]:
!cat server/package.json | grep mongoose

    "mongoose": "^4.13.9",


#### Add the Mongoose connection
Add the following to the top of `server/app.js`.

```javascript
var mongoose = require('mongoose');
mongoose.connect('mongodb://localhost:27017/nintendo');
var db = mongoose.connection;
db.on("error", console.error.bind(console, "Connection error"));
db.once("open", function(callback){
  console.log("Connection successful")
});
```

In [55]:
%%file server/app.js
var express = require('express');
var path = require('path');
var favicon = require('serve-favicon');
var logger = require('morgan');
var cookieParser = require('cookie-parser');
var bodyParser = require('body-parser');
// Mongoose connection
var mongoose = require('mongoose');
mongoose.connect('mongodb://localhost:27017/nintendo');
var db = mongoose.connection;
db.on("error", console.error.bind(console, "Connection error"));
db.once("open", function(callback){
  console.log("Connection successful")
});

var index = require('./routes/index');
var users = require('./routes/users');

var app = express();

// view engine setup
app.set('views', path.join(__dirname, 'views'));
app.set('view engine', 'ejs');

// uncomment after placing your favicon in /public
//app.use(favicon(path.join(__dirname, 'public', 'favicon.ico')));
app.use(logger('dev'));
app.use(bodyParser.json());
app.use(bodyParser.urlencoded({ extended: false }));
app.use(cookieParser());
app.use(express.static(path.join(__dirname, 'public')));

app.use('/', index);
app.use('/users', users);

// catch 404 and forward to error handler
app.use(function(req, res, next) {
  var err = new Error('Not Found');
  err.status = 404;
  next(err);
});

// error handler
app.use(function(err, req, res, next) {
  // set locals, only providing error in development
  res.locals.message = err.message;
  res.locals.error = req.app.get('env') === 'development' ? err : {};

  // render the error page
  res.status(err.status || 500);
  res.render('error');
});

module.exports = app;

Overwriting server/app.js


In [56]:
!head -n 20 server/app.js

var express = require('express');
var path = require('path');
var favicon = require('serve-favicon');
var logger = require('morgan');
var cookieParser = require('cookie-parser');
var bodyParser = require('body-parser');
// Mongoose connection
var mongoose = require('mongoose');
mongoose.connect('mongodb://localhost:27017/nintendo');
var db = mongoose.connection;
db.on("error", console.error.bind(console, "Connection error"));
db.once("open", function(callback){
  console.log("Connection successful")
});

var index = require('./routes/index');
var users = require('./routes/users');

var app = express();



#### Check if MongoDB initializes

In [57]:
import subprocess
proc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)

In [58]:
import time

time.sleep(5)
for line in proc.stdout:
    print(str(line))
    if 'Connection successful' in str(line):
        print("Success!")
        break

b'\n'
b'> server@0.0.0 start /Users/jitsejan/code/pelican-blog/content/notebooks/server\n'
b'> node ./bin/www\n'
b'\n'
b'Connection successful\n'
Success!


In [60]:
process=!lsof -i:3000 -t
expressid = int(process[0])
!kill -9 $expressid

### Create new model
Create the Mongoose model for the games that we gathered in the earlier step.

In [61]:
!mkdir server/models

In [62]:
%%file server/models/game.js
var mongoose = require("mongoose");
var Schema = mongoose.Schema;

var GameSchema = new Schema({
  console: {
    type: String
  },
  name: {
    type: String
  },
  image: {
    type: String
  },
  sales: {
    type: String
  }
});

module.exports = mongoose.model("Game", GameSchema);

Writing server/models/game.js


### Adding new route
Create the route to access the data of the games. In the scaffold we already have the `index.js` and `users.js`, so lets create the `games.js` to setup the routes for the new pages.

#### Existing routes

In [63]:
!cat server/routes/index.js

var express = require('express');
var router = express.Router();

/* GET home page. */
router.get('/', function(req, res, next) {
  res.render('index', { title: 'Express' });
});

module.exports = router;


In [64]:
!cat server/routes/users.js

var express = require('express');
var router = express.Router();

/* GET users listing. */
router.get('/', function(req, res, next) {
  res.send('respond with a resource');
});

module.exports = router;


#### Add the route requirement
Add the following to `server/app.js`:

```javascript
...
var consoles = require('./routes/games.js');
...
app.use('/games', games);
...
```

With `sed` we can insert text on a certain position in a file.

In [80]:
!sed -i "18i var games = require('./routes/games');" server/app.js

sed: 1: "server/app.js": unterminated substitute in regular expression


Enable the routes in the app:

In [67]:
!sed -i "36i app.use('/games', games);" server/app.js

sed: 1: "server/app.js": unterminated substitute in regular expression


and create the route file `server/routes/games.js`:

In [ ]:
%%file server/routes/games.js
var express = require('express');
var router = express.Router();
var Game = require("../models/game");

router.get('/', (req, res) => {
  Game.find({}, '', function (error, games){
    if (error) { game.error(error); }
    res.send({
      games: games
    })
  }).sort({_id:-1})
})

module.exports = router;

Start the server and verify the new route which should return a JSON object.

In [ ]:
import json
import requests
import time
proc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)
time.sleep(5)
resp = requests.get('http://localhost:3000/games').json()
print(json.dumps(resp, indent=4))

Lets grab the ID of the first game of the data to verify the route in the next step.

In [ ]:
game_id = resp['games'][0]['_id']
game_id

Kill the application again.

In [ ]:
process=!fuser 3000/tcp | awk '{print $1}'
expressid = int(process[1])
!kill -9 $expressid

Add another route to the `games.js` to get information for a single game.

```javascript
router.get('/:id', (req, res) => {
  var db = req.db;
  Game.findById(req.params.id, '', function (error, game) {
    if (error) { console.error(error); }
    res.send(game)
  })
})
```


In [ ]:
%%file server/routes/games.js
var express = require('express');
var router = express.Router();
var Game = require("../models/game");

router.get('/', (req, res) => {
  Game.find({}, '', function (error, games){
    if (error) { game.error(error); }
    res.send({
      games: games
    })
  }).sort({_id:-1})
})
router.get('/:id', (req, res) => {
  var db = req.db;
  Game.findById(req.params.id, '', function (error, game) {
    if (error) { console.error(error); }
    res.send(game)
  })
})

module.exports = router;

#### Verify the game detail route
As a final step to verify the API we check if we can get the detailed information for the game ID we saved in the previous step.

In [ ]:
import time
proc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)
time.sleep(5)
resp = requests.get('http://localhost:3000/games/'+game_id).json()
print(json.dumps(resp, indent=4))

In [ ]:
process=!fuser 3000/tcp | awk '{print $1}'
expressid = int(process[1])
!kill -9 $expressid

### Conclusion
For the back-end we have created an API using Mongoose and ExpressJS with the following two routes:
* All games
* Game detail
<hr/>

<img src="logos/vuejs.PNG" class="logo" />
## Front-end
The next step is to create the front-end that can talk to the back-end and visualize the information. As a Javascript framework we are going to use VueJS. We will install the Vue [CLI](https://github.com/vuejs/vue-cli).

#### Create the client folder and install Vue

In [ ]:
!rm -rf client

In [ ]:
!mkdir client && cd client && npm init -y && npm install vue-cli --save

In [ ]:
ls -la client

#### Check version of Vue

In [ ]:
!cd client && npm vue --version

### Create the scaffold
To create the scaffold we will use the `vue-cli`. However, this requires us to give direct input to the command line which is tricky because we need to answer different questions when creating the scaffold using the tool. The easiest way to automate this task is to use the Linux tool [Expect](https://linux.die.net/man/1/expect). Make sure the tool is installed on your system. 

```bash
$ sudo apt-get install expect -y
```

Note: if you do not want to use this trick, you can also use the external terminal and run

```bash
$ vue init webpack
```

in the client directory or simply clone the [repository](https://github.com/vuejs-templates/webpack) to your local file system.

The following script contains the answers for the prompts created by the `vue init`. 

In [ ]:
%%file client/init_expect_script.sh
#!/usr/bin/expect -f
spawn npx vue init webpack

expect "Generate project in current directory?" { send "Y\r" }
expect "Project name"
send "vueclient\r"
expect "Project description"
send "An experiment with Jupyter and MEVN\r"
expect "Author"
send "Jupyter\r\n"
expect "Vue build"
send "\r\n"
expect "Install vue-router?"
send "Y\r"
expect "Use ESLint to lint your code?"
send "Y\r"
expect "Pick an ESLint preset"
send "\r\n"
expect "Set up unit tests"
send "Y\r"
expect "Pick a test runner"
send "\r\n"
expect "Setup e2e tests with Nightwatch?"
send "Y\r"
expect "Should we run `npm install` for you after the project has been created? (recommended)"
send "\r\n"
interact

In [ ]:
!chmod 755 client/init_expect_script.sh

In [ ]:
!cd client && ./init_expect_script.sh

In [ ]:
ls client

#### Temporary fix
There seems to be an [issue](https://github.com/webpack/webpack-dev-server/issues/1259) with webpack which disables Node to start the application. For now the version of `webpack-dev-server` 2.9.7 seems to not throw any errors.

In [ ]:
!cd client && npm install webpack-dev-server@2.9.7 --save-dev

In [ ]:
import time
proc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
time.sleep(10)

### Check the default page
The default page of Vue should be available on port 8080. Lets use [Selenium](http://www.seleniumhq.org/) to visit the webpage and make a screenshot.

#### Setup Selenium
First create a Docker instance that runs Selenium to avoid the cumbersome installation on the local Linux machine.

In [ ]:
selenium_running = False
for cntr in docker_client.containers.list():
    if 'selenium' in cntr.attrs['Config']['Image']:
        selenium_running = True
        container = cntr
if selenium_running is False:
    container = docker_client.containers.run("selenium/standalone-chrome:latest", name='selenium', ports={'4444': '4444'}, detach=True)

In [ ]:
time.sleep(10)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

driver = webdriver.Remote("http://localhost:4444/wd/hub", DesiredCapabilities.CHROME)
driver.get('http://dev.jitsejan.com:8080')
driver.save_screenshot('homepage.png')

<img src="homepage.png" class="screenshot" />

Kill the process on port 8080 after verifying the front page.

In [ ]:
process=!fuser 8080/tcp | awk '{print $1}'
vueid = int(process[1])
!kill -9 $vueid

As we can see from the screenshot, we cannot access the page because `webpack` has changed some configuration. In order to circument this we need to add the host to the script in `package.json` and change

```javascript
    "dev": "webpack-dev-server --inline --progress --config build/webpack.dev.conf.js",
```

to

```javascript
    "dev": "webpack-dev-server --inline --progress --config build/webpack.dev.conf.js --host 0.0.0.0",
```

which can be done with `sed`:

In [ ]:
!sed -i '/dev.*--inline/s/conf.js/conf.js --host 0.0.0.0/' client/package.json

In [ ]:
cat client/package.json | grep dev

Additionally we need to disable the host check in `build/webpack.dev.conf.js` by adding 

```javascript
disableHostCheck: true
```

to the `devServer`.

In [ ]:
!sed -i "26i\ \ \ \ disableHostCheck: true," client/build/webpack.dev.conf.js

In [ ]:
!grep -C3 disableHostCheck client/build/webpack.dev.conf.js --color=auto

#### Retry

In [ ]:
proc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
time.sleep(10)

In [ ]:
driver = webdriver.Remote("http://localhost:4444/wd/hub", DesiredCapabilities.CHROME)
driver.get('http://dev.jitsejan.com:8080')
driver.get_screenshot_as_file('homepage_success.png')

<img src='homepage_success.png' class="screenshot" />

In [ ]:
process=!fuser 8080/tcp | awk '{print $1}'
vueid = int(process[1])
!kill -9 $vueid

## Create games page

In [ ]:
ls client/src/components/

### Create the component

In [ ]:
%%file client/src/components/Games.vue
<template>
  <div class="games">
    This page will list all the games.
  </div>
</template>

<script>
export default {
  name: 'Games',
  data () {
    return {}
  }
}
</script>

In [ ]:
ls client/src/components/

### Create the route

In [ ]:
cat client/src/router/index.js

Add the import of the Games component to the router file.

```javascript
import Games from '@/components/Games'
```

and add the path to the games page indicating to which component the page will link.

```javascript
...
    {
      path: '/games',
      name: 'Games',
      component: Games
    }
...
```

In [ ]:
%%file client/src/router/index.js
import Vue from 'vue'
import Router from 'vue-router'
import HelloWorld from '@/components/HelloWorld'
import Games from '@/components/Games'

Vue.use(Router)

export default new Router({
  routes: [
    {
      path: '/',
      name: 'HelloWorld',
      component: HelloWorld
    },
    {
      path: '/games',
      name: 'Games',
      component: Games
    }
  ]
})



#### Verify the new route

In [ ]:
proc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
time.sleep(10)

driver = webdriver.Remote("http://localhost:4444/wd/hub", DesiredCapabilities.CHROME)
driver.get('http://dev.jitsejan.com:8080/ui/#/games')
driver.get_screenshot_as_file('new_route.png')

process=!fuser 8080/tcp | awk '{print $1}'
vueid = int(process[1])
!kill -9 $vueid

<img src="new_route.png" class="screenshot" />

### Connect to back-end
We have the route complete, time to connect to the data from the database. For this we will use [axios](https://github.com/axios/axios).
#### Install axios

In [ ]:
!cd client && npm install --save axios

#### Setup connection with back-end
Create a `services` folder that will contain the API Javascript files.

In [ ]:
mkdir -p client/src/services

In [ ]:
%%file client/src/services/api.js
import axios from 'axios'

export default() => {
  return axios.create({
    baseURL: `http://dev.jitsejan.com:3000`
  })
}



#### Create service to retrieve the games

In [ ]:
%%file client/src/services/GamesService.js
import api from '@/services/api'

export default {
  fetchGames () {
    return api().get('games')
  }
}



#### Add the service to the Games component
This component shows all the games available in the database. Note that this is the file that should be updated if you would like a more fancy layout, for example by adding [Bootstrap](https://getbootstrap.com/) or other frameworks.

In [ ]:
%%file client/src/components/Games.vue
<template>
  <div class="games">
    <div class="games-div" v-for="game in games" :key="game._id">
      <p>
        <span><b>{{ game.name }}</b></span><br />
        <span>{{ game.console }}</span><br />
        <span>{{ game.sales }}</span>
        <a :href="'/ui/#/games/' + game._id">Details</a>
      </p>
    </div>
  </div>
</template>

<script>
import GamesService from '@/services/GamesService'

export default {
  name: 'Games',
  data () {
    return {
      games: []
    }
  },
  mounted () {
    this.getGames()
  },
  methods: {
    async getGames () {
      const response = await GamesService.fetchGames()
      this.games = response.data.games
    }
  }
}
</script>



Start both the Express back-end and the Vue front-end to verify if the data from the API is retrieved with the updated component.

Small trick: by setting the logging preferences when starting the webdriver with Selenium, we can actually retrieve the logs you would normally see with the developer tools in the browser.

In [ ]:
vueproc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
expressproc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)

time.sleep(10)

desired = DesiredCapabilities.CHROME
desired ['loggingPrefs'] = { 'browser':'ALL' }
        
driver = webdriver.Remote("http://localhost:4444/wd/hub", desired_capabilities=desired)
driver.get('http://dev.jitsejan.com:8080/ui/#/games')
driver.get_screenshot_as_file('new_route_with_data.png')

vueprocess=!fuser 8080/tcp | awk '{print $1}'
vueid = int(vueprocess[1])
!kill -9 $vueid

expressprocess=!fuser 3000/tcp | awk '{print $1}'
expressid = int(expressprocess[1])
!kill -9 $expressid

In [ ]:
logs = driver.get_log('browser')
logs

<img src="new_route_with_data.png" class="screenshot" />

As we can see in the logs, we have an issue accessing the back-end because of the Access-Control-Allow-Origin limitation. To circumvent this, we need to make sure that we are allowed to access the API; hece install the package [Cors](https://www.npmjs.com/package/cors) and enable it in the Express application.

In [ ]:
!cd server && npm install --save cors

Add the requirement for the cors package and enable it in the Express app by adding the following to `server/app.js`.

```javascript
...
var cors = require('cors')
...
app.use(cors())
...
```

In [ ]:
%%file server/app.js
var express = require('express');
var path = require('path');
var favicon = require('serve-favicon');
var logger = require('morgan');
var cookieParser = require('cookie-parser');
var bodyParser = require('body-parser');
// Mongoose connection
var mongoose = require('mongoose');
mongoose.connect('mongodb://localhost:27017/nintendo');
var db = mongoose.connection;
db.on("error", console.error.bind(console, "Connection error"));
db.once("openUri", function(callback){
  console.log("Connection successful")
});
var cors = require("cors")

var index = require('./routes/index');
var users = require('./routes/users');
var games = require('./routes/games');

var app = express();
app.use(cors());

// view engine setup
app.set('views', path.join(__dirname, 'views'));
app.set('view engine', 'ejs');

// uncomment after placing your favicon in /public
//app.use(favicon(path.join(__dirname, 'public', 'favicon.ico')));
app.use(logger('dev'));
app.use(bodyParser.json());
app.use(bodyParser.urlencoded({ extended: false }));
app.use(cookieParser());
app.use(express.static(path.join(__dirname, 'public')));

app.use('/', index);
app.use('/users', users);
app.use('/games', games);

// catch 404 and forward to error handler
app.use(function(req, res, next) {
  var err = new Error('Not Found');
  err.status = 404;
  next(err);
});

// error handler
app.use(function(err, req, res, next) {
  // set locals, only providing error in development
  res.locals.message = err.message;
  res.locals.error = req.app.get('env') === 'development' ? err : {};

  // render the error page
  res.status(err.status || 500);
  res.render('error');
});

module.exports = app;

Now we are connected and allowed to retrieve the data. We use Selenium and wait for the `games-div` to appear.

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

vueproc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
expressproc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)

time.sleep(10)

desired = DesiredCapabilities.CHROME
desired ['loggingPrefs'] = { 'browser':'ALL' }
        
driver = webdriver.Remote("http://localhost:4444/wd/hub", desired_capabilities=desired)
driver.get('http://dev.jitsejan.com:8080/ui/#/games')
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'games-div')))
driver.refresh()
driver.save_screenshot('allgames.png')

<img src="allgames.png" class="screenshot"/>

#### Clean up

In [ ]:
driver.quit()

vueprocess=!fuser 8080/tcp | awk '{print $1}'
vueid = int(vueprocess[1])
!kill -9 $vueid

expressprocess=!fuser 3000/tcp | awk '{print $1}'
expressid = int(expressprocess[1])
!kill -9 $expressid

#### Create detail page for the game
Create the component and connect to the API by importing the service and execute the promise. Retrieving the data for a game can also be solved by extending the `GamesServices.js` we created earlier by adding another fetch function, but since every problem can be solved in different ways, I chose to use the method as shown in the following file.

In [ ]:
!cp -r images client/static

In [ ]:
!ls client/static/images/

In [ ]:
%%file client/src/components/GamesDetail.vue
<template>
  <div class="games-detail">
    <h1>Details for {{ data.name }}</h1>
    <img :src="'/static/' + data.image" /><br/>
    Console: {{ data.console }}<br/>
    Sales: {{ data.sales}} million<br/>
  </div>
</template>

<script>
import api from '@/services/api'

export default {
  name: 'GamesDetail',
  data () {
    return {
      data: ''
    }
  },
  created () {
    api().get('games/' + this.$route.params.id)
      .then(response => {
        this.data = response.data
      })
      .catch(e => {
        this.errors.push(e)
      })
  }
}
</script>



Add the route for the detail page by adding the following to `client/src/router/index.js`:
    
```javascript
...
import GamesDetail from '@/components/GamesDetail'
...
    {
      path: '/games/:id',
      name: 'GamesDetail',
      component: GamesDetail
    }
```

In [ ]:
%%file client/src/router/index.js
import Vue from 'vue'
import Router from 'vue-router'
import HelloWorld from '@/components/HelloWorld'
import Games from '@/components/Games'
import GamesDetail from '@/components/GamesDetail'

Vue.use(Router)

export default new Router({
  routes: [
    {
      path: '/',
      name: 'HelloWorld',
      component: HelloWorld
    },
    {
      path: '/games',
      name: 'Games',
      component: Games
    },
    {
      path: '/games/:id',
      name: 'GamesDetail',
      component: GamesDetail
    }
  ]
})



Start the processes

In [ ]:
vueproc = subprocess.Popen("cd client && npm run dev &", shell=True, stdout=subprocess.PIPE)
expressproc = subprocess.Popen("cd server && npm start &", shell=True, stdout=subprocess.PIPE)
time.sleep(10)

Verify the detail page for a given id.

In [ ]:
desired = DesiredCapabilities.CHROME
desired ['loggingPrefs'] = { 'browser':'ALL' }
        
driver = webdriver.Remote("http://localhost:4444/wd/hub", desired_capabilities=desired)
driver.get('http://dev.jitsejan.com:8080/ui/#/games/5a5bd24b48f1a5314a089b80')
driver.save_screenshot('gamedetail.png')

<img class="screenshot" src="gamedetail.png"/>

In [ ]:
vueprocess=!fuser 8080/tcp | awk '{print $1}'
vueid = int(vueprocess[1])
!kill -9 $vueid

expressprocess=!fuser 3000/tcp | awk '{print $1}'
expressid = int(expressprocess[1])
!kill -9 $expressid

### Finalize
To wrap this notebook up, lets update the generic `App.vue` to make it easier to navigate between the pages by adding a menu to `client/src/App.vue`.

In [ ]:
%%file client/src/App.vue
<template>
  <div id="app">
    <router-link :to="{ name: 'HelloWorld' }">Home</router-link>
    <router-link :to="{ name: 'Games'}">Games</router-link>
    <router-view/>
  </div>
</template>

<script>
export default {
  name: 'app'
}
</script>

<style>
#app {
  font-family: 'Avenir', Helvetica, Arial, sans-serif;
  -webkit-font-smoothing: antialiased;
  -moz-osx-font-smoothing: grayscale;
  text-align: center;
  color: #2c3e50;
  margin-top: 60px;
}
</style>



## Future work
In this example application I did not use any fancy styling or advanced HTML structures to make the website more appealing. Additionally, the API is only used to retrieve data, while obviously it could also used to create data, update it or delete it (CRUD), but for the scope of this notebook I kept it to a minimum. As a third improvement you could choose to put everything of this notebook in one Docker compose file, where MongoDB, Selenium, ExpressJS and VueJS are all containerized, but since I am not planning to port this application to another server, it is fine to simply install the files locally and remove them again after the notebook has finished.